In [2]:
import numpy as np
import math as m
from random import shuffle
from time import sleep
import matplotlib.pyplot as plt
from IPython.display import clear_output
import sys

In [3]:
# Variable Initialization
totalHouses = 5
houses = [] 
animationSpeed = 0
houseRank = {
            1 : [],
            2 : [],
            3 : []
            }

# GA variables
population, fitness = [],[]
popSize = 500
generationNumber = 0

recordDistance, currentRecord = m.inf,m.inf
bestEver, currentBest = [],[]
statusP = 0
mutationRate = 0.01

loopBreaker = 0

In [4]:
#Genetic Algorithm
def calculateFitness():
    global houses, popSize, populatuion, recordDistance, bestEver, currentRecord, currentBest, fitness
    
    currentRecord = m.inf
    for i in range(popSize):
        d = calcDistance(houses, population[i])
        if d < recordDistance:
            recordDistance = d
            bestEver = population[i]
        if d < currentRecord:
            currentRecord = d
            currentBest = population[i]
        fitness.append(1 / (m.pow(d, 8) + 1))

def normalizeFitness():
    _sum = sum(fitness)
    for i in range(len(fitness)):
        fitness[i] = fitness[i] / _sum

def nextGeneration():
    global population, mutationRate, generationNumber
    
    newPopulation = []
    orderA, orderB, order = [], [], []
    for i in range(len(population)):
        orderA = pickOne(population , fitness)
        orderB = pickOne(population , fitness)
        order = crossOver(orderA, orderB)
        mutate(order)
        newPopulation.append(order)
    population = list(newPopulation)
    del(newPopulation)
    generationNumber += 1

def pickOne(lst, prob):
    index = 0
    r = np.random.rand(1)[0]
    while r>0:
        r = r - prob[index]
        index += 1
    index -= 1
    return list(lst[index])

def crossOver(orderA, orderB):
    start = np.random.randint(len(orderA)-1)
    end = np.random.randint(start+1, len(orderA))
    neworder = orderA[start:end]
    for i in range(len(orderB)):
        house = orderB[i]
        if house not in neworder:
            neworder.append(house)
    return neworder

def mutate(order):
    global mutationRate
    
    for i in range(totalHouses):
        if (np.random.rand(1)[0]) < mutationRate:
            indexA = np.random.randint(len(order))
            indexB = (indexA + 1) % totalHouses
            swap(order, indexA, indexB)
#Genetic Algorithm

In [7]:
def variableReset(mrt,pops,tHouses):
    global totalHouses, houses, animationSpeed, population, fitness, popSize, generationNumber, recordDistance, currentRecord, bestEver, currentBest, statusP, mutationRate
    
    totalHouses = tHouses
    houses = [] 
    animationSpeed = 0
    
    # GA variables
    population, fitness = [],[]
    popSize = pops
    generationNumber = 0

    recordDistance, currentRecord = m.inf,m.inf
    bestEver, currentBest = [],[]
    statusP = 0
    mutationRate = mrt

def setup():
    global totalHouses, houses, population, popSize
    
    variableReset(mutationRate, popSize, totalHouses)
    order = []
    latitude, longitude, rank = 0,0,0
    
    for i in range(totalHouses):
        # houses.append((np.random.randint(10, high=1000, size=2)).tolist())
        print(f'Entry for house {i+1}')
        latitude = float(input("\tLatitude : "))
        longitude = float(input("\tLongitude : "))
        houses.append([latitude, longitude])
        houses[i].append(i+1)
        order.append(i)
        rank = int(input("\tRank: "))
        if rank in houseRank:
            houseRank(rank).append(i)
    
    for i in range(popSize):
        population.append(list(order))
        shuffle(population[i])
        
def draw():
    global generationNumber 
    
    calculateFitness()
    normalizeFitness()
    nextGeneration()
    
    graphingHouses()
    print(f'Generation: {generationNumber}')

def graphingHouses():
    global totalHouses, houses, recordDistance, bestEver, currentBest, currentRecord, animationSpeed
    
    xList, yList, _x, _y = [],[],[],[]
    plt.style.use('dark_background')
    plt.subplots(figsize=(20, 10))
    fig = plt.subplot(1,2,1)

    for i in bestEver: 
        x=houses[i][0]
        y=houses[i][1]
        _x.append(x)
        _y.append(y)
    # Plotting the graph of the shortest path
    plt.plot(_x,_y,'ro',ms='6')
    plt.plot(_x, _y, color='pink',ms='5')
    plt.title("All Time Best\nTotal distance = {0:.5f} units\nVisiting House sequence:{1}"
              .format(recordDistance,str([(i+1) for i in bestEver])))
    plt.ylim(0,1030)
    plt.xlim(0,1030)
    plt.ylabel('Latitude')
    plt.xlabel('Longitude')
    plt.grid(color='grey', linestyle='-', linewidth='0.25')
    for i,j in enumerate(bestEver):
        plt.annotate('house {}'.format(houses[j][2]),xy=(_x[i]-15, _y[i]-10), color='y', fontsize=10)


    fig = plt.subplot(1,2,2)
    for i in currentBest: 
        x=houses[i][0]
        y=houses[i][1]
        xList.append(x)
        yList.append(y)
        plt.plot(x,y,'ro',ms='5')
        plt.annotate('house {}'.format(houses[i][2]),xy=(x-15, y-10), color='y', fontsize=10)
    plt.style.use('dark_background')
    plt.ylim(0,1030)
    plt.xlim(0,1030)
    plt.ylabel('Latitude')
    plt.xlabel('Longitude')
    plt.grid(color='grey', linestyle='-', linewidth='0.25')
    plt.plot(xList, yList, color='white')
    plt.title("Generation Best\nTotal distance = {0:.5f} units\nVisiting House sequence:{1}"
              .format(currentRecord,str([(i+1) for i in currentBest])))    

    plt.subplots_adjust(left=0.1,
                        bottom=0.1, 
                        right=0.9, 
                        top=0.9, 
                        wspace=0.4, 
                        hspace=0.4 )
    plt.show() # Displaying all the ploted graph(s)
    sleep(animationSpeed) # Setting the animation speed of the output
        
def swap(a, i ,j):
    a[i], a[j] = a[j], a[i]

def calcDistance(points, order):
    _sum = 0
    for i in range(len(order)-1):
        houseAindex = points[order[i]]
        houseBindex = points[order[i+1]]
        d = m.dist(houseAindex,houseBindex)
        _sum += d
    return _sum

In [8]:
setup()

Entry for house 1


	Latitude :  123.12
	Longitude :  123.3
	Rank:  1


TypeError: 'dict' object is not callable

In [ ]:
extension = 100
while True:
    draw()    
    if generationNumber >= extension:
        if loopBreaker == recordDistance:
            print(f"Shortest visiting sequence of houses is {bestEver}, with total diatnace to cover {recordDistance} units.\nProgram Completed successfully.")
            sys.exit()
        else:
            loopBreaker = recordDistance
            extension += 100 ;
    clear_output(wait=True)